# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: gpt2
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [32]:
!pip install transformers datasets evaluate accelerate peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 820.1 kB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [33]:
# TRAIN_TEST_SIZE = 25
TRAIN_TEST_SIZE = 250
from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = load_dataset("imdb")

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(TRAIN_TEST_SIZE))

ds.pop('unsupervised')
# Show the dataset|
ds


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
})

In [34]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
# GPT-2 Tokenizer doesn't have a padding token.
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    tokens = tokenizer(examples['text'], padding='max_length', truncation=True)
    return tokens

# Let's use a lambda function to tokenize all the examples
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)

# Inspect the available columns in the dataset
tokenized_ds

{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 250
 }),
 'test': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 250
 })}

In [35]:

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE":1},
)
model.config.pad_token_id = model.config.eos_token_id

# Freeze all the layers so only the classification head is trained.
for param in model.base_model.parameters():
    param.requires_grad = False


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=3e-1,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=10,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,23.394930,0.652000
2,No log,20.399174,0.676000
3,No log,13.715519,0.720000
4,No log,43.899517,0.624000
5,No log,19.214195,0.676000
6,No log,50.976048,0.540000
7,No log,44.113697,0.524000
8,18.494700,12.177135,0.748000
9,18.494700,13.404923,0.760000
10,18.494700,12.794985,0.700000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-63 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-126 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-189 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-252 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-315 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=630, training_loss=15.240864199683779, metrics={'train_runtime': 476.467, 'train_samples_per_second': 5.247, 'train_steps_per_second': 1.322, 'total_flos': 1306483752960000.0, 'train_loss': 15.240864199683779, 'epoch': 10.0})

In [37]:
trainer.evaluate()

{'eval_loss': 12.17713451385498,
 'eval_accuracy': 0.748,
 'eval_runtime': 21.7676,
 'eval_samples_per_second': 11.485,
 'eval_steps_per_second': 2.894,
 'epoch': 10.0}

In [38]:
def get_sentiment(sentence, model):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs.to(device='cuda'))
    prediction = outputs.logits.argmax(-1).item()
    return "positive" if prediction == 1 else "negative"


sentence = "I did not like this movie!"
print(get_sentiment(sentence, model))

negative


In [39]:
model.save_pretrained("gpt")

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [40]:
from peft import LoraConfig, TaskType
from peft import get_peft_model
from peft import AutoPeftModelForSequenceClassification

model_loaded = AutoModelForSequenceClassification.from_pretrained("gpt")

lora_config = LoraConfig(
fan_in_fan_out=True,
task_type=TaskType.SEQ_CLS
)

lora_model = get_peft_model(model_loaded, lora_config)
lora_model.config.pad_token_id = lora_model.config.eos_token_id

In [41]:
lora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()

In [42]:
# LoRA takes in "labels", not "label" so we need to rename the 
# training and testing sets
train_lora = tokenized_ds['train'].rename_column('label', 'labels')
test_lora = tokenized_ds['test'].rename_column('label', 'labels')

In [43]:

trainer_lora = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
trainer_lora.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,21.989771,0.484000
2,No log,13.098240,0.484000
3,No log,9.570612,0.516000
4,No log,3.367130,0.484000
5,No log,1.812402,0.484000
6,No log,4.117071,0.484000
7,No log,4.842762,0.484000
8,18.407800,1.471814,0.516000
9,18.407800,1.667329,0.516000
10,18.407800,0.778312,0.516000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-63 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-126 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-189 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-252 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-315 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-378 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/s

TrainOutput(global_step=630, training_loss=14.895753939189609, metrics={'train_runtime': 741.7244, 'train_samples_per_second': 3.371, 'train_steps_per_second': 0.849, 'total_flos': 1311037194240000.0, 'train_loss': 14.895753939189609, 'epoch': 10.0})

In [44]:
trainer_lora.evaluate()

{'eval_loss': 0.7783116698265076,
 'eval_accuracy': 0.516,
 'eval_runtime': 22.1683,
 'eval_samples_per_second': 11.277,
 'eval_steps_per_second': 2.842,
 'epoch': 10.0}

In [45]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [46]:
from peft import AutoPeftModelForSequenceClassification
lora_model_loaded = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")

In [47]:
# now we put the loaded model into a trainer so we can use the evaluate method and 
# compare it to the foundational model
trainer_lora_loaded = Trainer(
    model=lora_model_loaded,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [48]:
trainer_lora_loaded.evaluate()

{'eval_loss': 0.7783116698265076,
 'eval_accuracy': 0.516,
 'eval_runtime': 22.1801,
 'eval_samples_per_second': 11.271,
 'eval_steps_per_second': 2.84}

In [49]:
def get_sentiment(sentence, model):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs.to(device='cuda'))
    prediction = outputs.logits.argmax(-1).item()
    return "positive" if prediction == 1 else "negative"


In [50]:
sentence = "I dont like this movie!"
print(get_sentiment(sentence, lora_model_loaded.to(device='cuda')))

negative
